# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [4]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [6]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [7]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [9]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [10]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [14]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our sales team.

Action: Read a file's content
Action Input: {"file_path": "./instructions/DeepLearningAI_company_background.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions/DeepLearningAI_company_background.txt'.
 T

Thought: Let's search for information on key decision-makers at DeepLearningAI to include in the report.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI key decision-makers"} 


Search results: Title: Artificial Intelligence and Deep Learning for Decision Makers
Link: https://community.deeplearning.ai/t/artificial-intelligence-and-deep-learning-for-decision-makers-driving-smarter-business-strategies/881460
Snippet: These technologies empower leaders to: Enhance Forecasting and Planning: AI algorithms can analyze historical sales, market trends, and ...
---
Title: Courses - DeepLearning.AI
Link: https://www.deeplearning.ai/courses/
Snippet: Build a solid data analytics foundation using industry standard and AI tools to extract insights, make decisions, and solve real-world business problems.
---
Title: Deep Learning Specialization - DeepLearning.AI
Link: https://www.deeplearning.ai/courses/deep-learning-specialization/
Snippet: In the third course of the Deep

Thought: Let's search for information on key decision-makers at DeepLearningAI to include in the report.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI leadership team"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: Let's try another search query to find information on key decision-makers at DeepLearningAI.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI leadership team information"} 


Search results: Title: About - DeepLearning.AI
Link: https://www.deeplearning.ai/about/
Snippet: DeepLearning.AI is an education technology company that is empowering the global workforce to build an AI-powered future through world-class education.
---
Title: Andrew Ng - Founder of DeepLearning.AI - LinkedIn
Link: https://www.linkedin.com/in/andrewyng
Snippet: Andrew Ng - Founder of DeepLearning.AI; Managing General Partner of AI Fund; Exec Chairman of LandingAI | LinkedIn.
---


2026-01-19 10:35:44,027 - 139856086539136 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--ca2e41d7c851a08841e7c8984ccf16763c2c519052128765404801837054624f


Thought: I should read the content from the most relevant search results to gather detailed information about key decision-makers at DeepLearningAI.

Action: Read a file's content
Action Input: {"file_path": "https://www.linkedin.com/company/deeplearningai"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: 'https://www.linkedin.com/company/deeplearningai'.
 Tool Read a file's content accepts these inputs: Read a file's content(file_path: 'string') - A tool that can be used to read a file's content.

Thought: Let's search for information on key decision-makers at DeepLearningAI to include in the report.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI leadership team members LinkedIn"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: Let's try another search query to find information on key decision-makers at DeepLearningAI.

Action



> Entering new CrewAgentExecutor chain...
I need to craft personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng, the CEO. The emails should connect our solutions with their recent achievements and future goals, resonating with DeepLearningAI's company culture and values.

Action: Search the internet
Action Input: {"search_query": "Recent achievements and future goals of DeepLearningAI"} 


Search results: Title: Future of Deep Learning according to top AI Experts in 2026
Link: https://research.aimultiple.com/future-of-deep-learning/
Snippet: Deep learning currently delivers the best results for many AI applications. But there's debate about its ultimate potential.
---
Title: The Future of AI According to Today's Most Influential AI Figures
Link: https://medium.com/softtechas/the-future-of-ai-according-to-todays-most-influential-ai-figures-5cd294bd390e
Snippet: As AI becomes more visibly powerful, companies will naturally become more paranoid and colla

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Email Draft to Andrew Ng, CEO of DeepLearningAI:

Subject: Innovating Together: Enhancing AI Education with CrewAI

Dear Mr. Ng,

I hope this message finds you well. I am reaching out on behalf of CrewAI, a leading provider of cutting-edge solutions tailored to support organizations like DeepLearningAI in achieving their educational goals.

After conducting in-depth research on DeepLearningAI, I am truly impressed by the company's commitment to providing world-class AI education accessible to individuals globally. Your recent milestones and dedication to advancing AI education are truly commendable.

As we look towards the future of AI and deep learning, there is no doubt that the landscape is rapidly evolving. Our team at CrewAI is dedicated to innovating alongside industry leaders like DeepLearningAI to ensure that the workforce of tomorrow is equipped with the skills and knowledge needed to thrive in the AI sector.

Our solutions leverage cutting-edge tools and strategic analysis to enhance educational offerings, support personalized learning experiences, and address the evolving needs of the AI industry. By collaborating with DeepLearningAI, we believe that we can create enhanced educational experiences, innovative learning solutions, and a shared commitment to empowering the global workforce in the AI sector.

I would love the opportunity to discuss how CrewAI can support DeepLearningAI's mission and vision further. Please let me know a convenient time for a call or meeting to explore potential collaboration opportunities.

Thank you for your time, Mr. Ng. I look forward to the possibility of working together to shape the future of AI education.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

---
Note: Make sure to personalize the email with your name and contact information before sending it out.